In [1]:
print("Kernel working fine!")

Kernel working fine!


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [4]:
if GOOGLE_API_KEY == "":
    print("API key cannot be fetched!")
else:
    print("API key fetched!")

API key fetched!


In [5]:
from llama_index.core.readers import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.llms.gemini import Gemini
from IPython.display import display, Markdown
from llama_index.core import ServiceContext
from llama_index.core import StorageContext, load_index_from_storage
import google.generativeai as genai
from llama_index.embeddings.gemini import GeminiEmbedding

/Users/atharva7/Downloads/QA System/qasystem/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
genai.configure(api_key=GOOGLE_API_KEY)

In [7]:
for models in genai.list_models():
    if 'generateContent' in models.supported_generation_methods:
        print(models.name)

models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash-preview-04-17-thinking
models/gemini-2.5-pro-preview-05-06
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-fl

In [8]:
documents = SimpleDirectoryReader("../Data")

In [9]:
documents = documents.load_data()

In [10]:
gemini_embed_model = GeminiEmbedding(model_name='models/embedding-001')

/var/folders/8j/ythr_y152qx4ycdfpz80jjf80000gn/T/ipykernel_45919/3857009644.py:1: DeprecationWarning: Call to deprecated class GeminiEmbedding. (Should use `llama-index-embeddings-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/embeddings/google_genai/)
  gemini_embed_model = GeminiEmbedding(model_name='models/embedding-001')


In [11]:
model = Gemini(models='gemini-1.5-pro-latest', api_key=GOOGLE_API_KEY)

/var/folders/8j/ythr_y152qx4ycdfpz80jjf80000gn/T/ipykernel_45919/4206100260.py:1: DeprecationWarning: Call to deprecated class Gemini. (Should use `llama-index-llms-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/llm/google_genai/)
  model = Gemini(models='gemini-1.5-pro-latest', api_key=GOOGLE_API_KEY)


In [12]:
# Now we will configure the service context.
from llama_index.core import Settings
from llama_index.core.node_parser import SentenceSplitter

# Set the new configuration
Settings.llm = model 
Settings.embed_model = gemini_embed_model  
Settings.node_parser = SentenceSplitter(chunk_size=500, chunk_overlap=20)


In [13]:
index = VectorStoreIndex.from_documents(documents)

In [14]:
index

In [15]:
index.storage_context.persist()

In [16]:
query_engine = index.as_query_engine()

In [17]:
response = query_engine.query("What is meant by quantum physics?")

In [18]:
response

Response(response='Quantum mechanics refers to a system where the number of particles is fixed and the fields are continuous classical entities, but it is also often used to refer to the entire notion of quantum view.\n', source_nodes=[NodeWithScore(node=TextNode(id_='b26ca568-1d9d-4d58-960a-3d6af270cc64', embedding=None, metadata={'file_path': '/Users/atharva7/Downloads/QA System/Notebook/../Data/Quantum_PHY.txt', 'file_name': 'Quantum_PHY.txt', 'file_type': 'text/plain', 'file_size': 35270, 'creation_date': '2025-06-04', 'last_modified_date': '2025-06-04'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='b5c80d51-1ed4-40ee-853c-7dd4334e3212', node_type='4', metadata={'file_path': '/Users/atharva7/Download

In [19]:
print(response.response)

Quantum mechanics refers to a system where the number of particles is fixed and the fields are continuous classical entities, but it is also often used to refer to the entire notion of quantum view.



In [20]:
response2 = query_engine.query("What is the use of quantum physics in our day to day life?")

In [21]:
print(response2.response)

Quantum physics is used in lasers, transistors, electron microscopes, magnetic resonance imaging, light switches, and flash memory chips.



In [22]:
response3 = query_engine.query("What is a water bottle?")

In [23]:
print(response3.response)

I'm sorry, but the provided text does not contain information about water bottles.



In [25]:
from duckduckgo_search import DDGS

In [27]:
def web_search(query, max_results=3):
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=max_results)
        return [res['body'] for res in results if 'body' in res]

In [28]:
def smart_query(query):
    response = query_engine.query(query).response
    print("🔎 Internal Response:", response)

    fallback_phrases = [
        "does not contain", "i don't know", "i'm sorry", "not found", "no relevant", "no information"
    ]
    
    if any(phrase in response.lower() for phrase in fallback_phrases):
        print("⚠️ Internal knowledge insufficient. Searching web...")
        web_docs = web_search(query)
        if not web_docs:
            return "❌ No information found on the web either."
        
        # Feed web docs into the LLM
        combined_text = "\n".join(web_docs)
        web_response = model.complete(prompt=f"Answer the following based on:\n{combined_text}\n\nQ: {query}\nA:")
        return web_response

    return response

In [29]:
answer = smart_query("What is a water bottle?")

🔎 Internal Response: I'm sorry, but the provided text does not contain information about water bottles.

⚠️ Internal knowledge insufficient. Searching web...


In [30]:
print(answer)

Based on the provided information, a water bottle is:

*   A container used to hold liquids, mainly water, for transporting a drink.
*   A small bottle containing water for drinking.
*   A container for carrying drinking water.



In [31]:
question = smart_query("who was the founder of quantum physics?")
print(question)

🔎 Internal Response: Planck and Einstein initiated quantum physics.

Planck and Einstein initiated quantum physics.

